# Ditto Training Pipeline (Global Style)

This notebook sets up the environment and runs the Ditto training pipeline on the global style editing subset (videos/global_style1 & videos/global_style2) of Ditto-1M on Google Colab Pro.

## 1. Mount Google Drive
Mount your Google Drive to save checkpoints and access the dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2. Setup Environment
Clone the repository and install necessary dependencies.

In [ ]:
!git clone https://github.com/dhruv0000/ditto-vace-fork.git
%cd ditto-vace-fork
!pip install -r requirements.txt
!pip install -e .
!pip install accelerate

## 3. Data Setup
Download the global style dataset and metadata from Hugging Face.

In [ ]:
#@title Data Download/Setup (Global Style)
import os
from huggingface_hub import snapshot_download

dataset_root = "./Ditto-1M"

# Download metadata JSON and CSV for the global style task
snapshot_download(
    repo_id="QingyanBai/Ditto-1M",
    repo_type="dataset",
    local_dir=dataset_root,
    allow_patterns=[
        "training_metadata/global_style.json",
        "csvs_for_DiffSynth/global_style.csv",
    ],
)

# Download source and global style video archives
snapshot_download(
    repo_id="QingyanBai/Ditto-1M",
    repo_type="dataset",
    local_dir=dataset_root,
    allow_patterns=[
        "videos/source/*",
        "videos/global_style1/*",
        "videos/global_style2/*",
    ],
)

print("Metadata and video archives downloaded.")

# Optionally extract the split archives for global_style1 and global_style2
extract_archives = True  #@param {type:"boolean"}

if extract_archives:
    import glob

    cwd = os.getcwd()
    try:
        for subset in ["global_style1", "global_style2"]:
            subset_dir = os.path.join(dataset_root, "videos", subset)
            if not os.path.isdir(subset_dir):
                print(f"Directory {subset_dir} does not exist, skipping.")
                continue

            os.chdir(subset_dir)
            part_pattern = f"{subset}.tar.gz.*"
            part_files = sorted(glob.glob(part_pattern))
            if not part_files:
                print(f"No split archives found for {subset}, skipping extraction.")
                continue

            print(f"Extracting {subset} from {len(part_files)} parts...")
            os.system(f"cat {part_pattern} | tar -zxv")
    finally:
        os.chdir(cwd)

print("Data setup complete.")

## 4. Configuration
Configure the training parameters for the global style task. The default model is `Wan-AI/Wan2.1-VACE-1.3B`.

In [ ]:
#@title Training Configuration

dataset_base_path = "./Ditto-1M/videos" #@param {type:"string"}
dataset_metadata_path = "./Ditto-1M/csvs_for_DiffSynth/global_style.csv" #@param {type:"string"}
output_path = "/content/drive/MyDrive/exps/ditto" #@param {type:"string"}
model_id = "Wan-AI/Wan2.1-VACE-1.3B" #@param {type:"string"}
num_epochs = 5 #@param {type:"integer"}
learning_rate = "1e-4" #@param {type:"string"}

print(f"Configuration:")
print(f"  Dataset Base Path: {dataset_base_path}")
print(f"  Metadata Path: {dataset_metadata_path}")
print(f"  Output Path: {output_path}")
print(f"  Model ID: {model_id}")
print(f"  Epochs: {num_epochs}")
print(f"  Learning Rate: {learning_rate}")

## 5. Run Training
Execute the training script with the configured parameters.

In [ ]:
!chmod +x train.sh
!./train.sh \
  --dataset_base_path "{dataset_base_path}" \
  --dataset_metadata_path "{dataset_metadata_path}" \
  --output_path "{output_path}" \
  --model_id "{model_id}" \
  --num_epochs "{num_epochs}" \
  --learning_rate "{learning_rate}"